In [0]:
from pyspark.sql import functions as F

In [0]:
data = [
    ("Ford F 150", "Blue", "Truck"),
    ("Jeep Wrangler", "Red", "SUV"),
    ("Toyota Highlander", "Silver", "SUV"),
    ("Honda Accord", "Black", "Sedan"),
    ("Chevrolet Tahoe", "White", "SUV"),
    ("Nissan Frontier", "Gray", "Truck"),
    ("Subaru Outback", "Green", "Wagon"),
    ("BMW X5", "Blue", "SUV"),
    ("Audi A4", "Red", "Sedan"),
    ("Ram 1500", "Maroon", "Truck"),
    ("Kia Sorento", "Silver", "SUV"),
    ("Hyundai Elantra", "Yellow", "Sedan"),
]


df = spark.createDataFrame(data, ["car", "color", "style"])
df.write.mode("overwrite").saveAsTable("workspace.bundles.bronze_cars")

#verify 
display(df)

In [0]:
silver_df = df.withColumn(
    "car_size",
    F.when(F.col("style") == "Truck", "Large")
     .when(F.col("style") == "SUV", "Medum")
     .otherwise("Small")
)
# Write to silver
silver_df.write.mode("overwrite").saveAsTable("workspace.bundles.silver_cars")

# Verify
display(spark.table("workspace.bundles.silver_cars"))

In [0]:
from pyspark.sql import functions as F

# Read from silver
silver_df = spark.table("workspace.bundles.silver_cars")

# Aggregate counts by car_size
gold_df = silver_df.groupBy("car_size").agg(
    F.count("*").alias("car_count")
)

# Write to gold
gold_df.write.mode("overwrite").saveAsTable("workspace.bundles.gold_cars")

# Optional view
display(gold_df)
